# Comparison between learning sensorimotor models 
## Part 1 : non parametric models

In this notebook, we compare the different sensorimotor models used in explauto in order to help the reader to use the most appropriate model depending on his aim. The notebook is split in two parts : non parametrics models (this notebook) and models based on gaussian mixtures (coming soon). There is also a tutorial explaining how to use the [SensorimotorModel abstract class](http://flowersteam.github.io/explauto/explauto.sensorimotormodel.html#explauto.sensorimotor_model.sensorimotor_model.SensorimotorModel) ([learning_sensorimotor_models](http://nbviewer.ipython.org/github/flowersteam/explauto/blob/master/notebook/learning_sensorimotor_models.ipynb)).

As explained in the [Explauto introduction](http://flowersteam.github.io/explauto/about.html), an important challenge in Developmental Robotics is how robots can efficiently learn sensorimotor mappings by experience, i.e. the mappings between the motor actions they make and the sensory effects they produce. This can be a robot learning how arm movements make physical objects move, or how movements of a virtual vocal tract modulates vocalization sounds.

Let's begin with defining a simple environment that will be used to test the sensorimotor models.

In [1]:
from explauto import Environment
environment = Environment.from_configuration('simple_arm', 'low_dimensional')

### Sensorimotor models introduction

In Explauto, a sensorimotor model implements both the iterative learning process from sensorimotor experience, i.e. from the iterative collection of $(m, s)$ pairs by interaction with the environment, and the use of the resulting internal model to perform forward and inverse predictions (or any kind of general prediction between sensorimotor subspaces). 

Learning sensorimotor mappings involves machine learning algorithms, for which Explauto provides a unified interface through the [SensorimotorModel abstract class](http://flowersteam.github.io/explauto/explauto.sensorimotormodel.html#explauto.sensorimotor_model.sensorimotor_model.SensorimotorModel). 

Using the simple arm environment above, it allows to iteratively learn a sensorimotor model which will be able to:
* infer the position of the end-effector from a given motor command, what is called *forward prediction*,
* infer the motor command allowing to reach a particular end-effector position, what is called *inverse prediction*.
* update online from sensorimotor experience

### Decision tree

In order to use the most appropriate model, there are several questions that have to be asked before using the following decision tree. The following paragraphs focus on these questions.

*Add the decision tree*

### Non parametric or gaussian mixture models ? 

Each of non parametric model is currently based on the nearest neigbhor look up. They are non paramtric models because they don't rely on assomptions that the data are drown from a given probability while gassian mixture models (GMM) assumes that the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. Thus, the last model can be used only with gaussian distributed data. Readers in this case should refer to the follwing tutorial : *coming soom* .

### Presentation of the different non parametric models

Available non parametric sensorimotor models in Explauto can be accessed using: 

In [2]:
from explauto.sensorimotor_model import sensorimotor_models
print 'Available sensorimotor models: {}'.format(sensorimotor_models.keys())

Available sensorimotor models: ['LWLR-BFGS', 'nearest_neighbor', 'WNN', 'LWLR-CMAES']


These 4 models are a combination of a forward and an inverse model:
* the **nearest neighbor** model searches the nearest point of a given input ($m$ or $s$) in the dataset and returns its corresponding value (respectively $s$ or $m$)
* the **WNN or weighted nearest neighbor** model searches the $k$ nearest points of a given input ($m$ or $s$) in the dataset and returns the average of the $k$ corresponding values (respectively $m$ or $s$)
* the **LWLR or The Locally Weigthed Linear Regression (LWLR)** offers two inverses model for the same forward one. The forward computes a linear regression of the $k$ nearest neighbors of $m$ and find the requested $s$ with the given $m$ based on that regression. Both inverse models are optimisation algorithm that minimize the error  $e(m) = ||LWLR(m) - s_g||^2$  where $s_g$ is the goal, $LWLR$ is the forward model LWLR, and $m$ is the motor command to be infered. Both **LWLR-BFGS** and **LWLR-CMAES** use LWLR forward model to predict $s$ but LWLR-BFGF uses Broyden–Fletcher–Goldfarb–Shanno algorithm and LWLR-CMAES covariance matrix adaptation which are going to be explained later.
Let's see which of these 8 *(2 * 4)* possibilities is the best

### Eploit or explore mode ?

All the non-parametric sensorimotor models have two operating modes: "explore" and "exploit".

In the "explore" mode, when the agent asks for the exact inverse prediction $m$ of a goal $s_g$, $m$ will be perturbated with some gaussian exploration noise in order to allow the agent to explore new motor commands. The sensorimotor models thus have a common parameter: sigma_explo_ratio=0.1 (default), which is the standard deviation of the gaussian noise, scaled depending of the motor domain size: if a motor value is bounded in [-2:2], then a sigma_explo_ratio of 0.1 will induce an exploration noise of (m_max - m_min) * sigma_explo_ratio = 0.4

In the "exploit" mode, no exploration noise is added. This mode is used for instance when evaluating the inverse model for comparison purposes.

*How to change it ? *

### Forward or inverse model ?

The forward model uses the dataset for the forward prediction computation, and the inverse model uses the forward model, or directly the dataset to perform inverse prediction. In other words, forward models predict $s_p$ given a $m$ that might have never been observed, using the dataset of observations $(m,s)$ and inverse models infer a motor command $m$ that should be able to reach a given goal $s_g$.  

In any case, it is possible to load the different library this way : 

In [8]:
from scipy import spatial
import numpy as np
import time

#Environment definition
from explauto.environment.environment import Environment
environment = Environment.from_configuration('simple_arm', 'mid_dimensional')

from explauto.sensorimotor_model import sensorimotor_models
from explauto import SensorimotorModel

### Forward models comparison
For each model : 
* Output descrptions
* Processing description
* When should it (or not) be used
* Time processing
* Distance between sg et s

#### Calculation comparison & computation

##### Nearest Neighbor forward model

To perform a forward prediction, the Nearest Neighbor model just looks in the dataset of tuples $(m, s)$ for the nearest neighbor of the given $m$ motor command, and returns its corresponding $s$.
The algorithm comes from scipy library : [scipy.spatial.KDTree.query](http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.spatial.KDTree.query.html) (with x = $s$, k=1, eps = 0, p = 2, radius = +inf). It returns distance and indexes of found nearest neighbors.
It works sufficiently well in different typical robotic applications but can be very long if the dataset's size exceed $10⁵$. 

Let's see how to use it : 

In [24]:
#Creation of the dataset
random_motors = environment.random_motors(n=1000)
    
sm_cls, sm_configs = sensorimotor_models['nearest_neighbor']
model = sm_cls(environment.conf, **sm_configs['default'])

for m in random_motors:
    s = environment.compute_sensori_effect(m)
    model.update(m, s)

In [25]:
#Test of the model
test = environment.random_motors(n=50)
distance = []
timer = []

for mTest in test :
    start = time.time()
    sTest_pred = model.forward_prediction(mTest)
    end = time.time()
    sTest_eff = environment.compute_sensori_effect(mTest)
    distance.append(spatial.distance.pdist([sTest_pred,sTest_eff])[0])
    timer.append(end - start)

In [26]:
# Distance between the predictive and effectiv value
comparison = {}
comparison['NN'] = [np.mean(distance), np.std(distance) ,max(distance),
                    np.mean(timer), np.std(timer), max(timer)]

##### Weighted Nearest Neighbor forward model

To perform a forward prediction of $m$, the Weighted Nearest Neighbor model looks at the $k$ (parameter) nearest neighbors of $m$ in the dataset, and returns the average of the $k$ corresponding $s$. This average is weighted by the distance to $m$ with a gaussian of standard deviation $\sigma$ (parameter). It is the same algorithm than for NN model with k = n the number of interesting neigbhors. ([scipy.spatial.KDTree.query](http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.spatial.KDTree.query.html)) It returns *How to choose the number of k? ? Can we change it? how does it compute ?*

In order to compute WNN, the process is exactly the same than for NN research but these lines :

In [10]:
sm_cls, sm_configs = sensorimotor_models['WNN']

In [34]:
comparison['WNN'] = [np.mean(distance), np.std(distance) ,max(distance),
                    np.mean(timer), np.std(timer), max(timer)]

##### Locally Weighted Linear Regression forward model

The LWLR computes a linear regression of the $k$ nearest neighbors of $m$ (thus a local regression), and find the requested $s$ with the given $m$ based on that regression. *Ewplain which method is used to compute linear regression*

In [30]:
sm_cls, sm_configs = sensorimotor_models['LWLR-BFGS']

In [32]:
comparison['LWLR'] = [np.mean(distance), np.std(distance) ,max(distance),
                    np.mean(timer), np.std(timer), max(timer)]

#### Distance between $s$ and $s_p$ comparison

If we look at comparisons values with a 1000 training $(m,s)$, we can easily notice that LWLR is closer to the goal on average than NN or WNN model. NN is really further than the two others which are not so different and the 3 standard deviation is equivalent. And the maximum distance can be sometimes very far away between the prediction and the goal for NN models. With a bigger dataset (10000), NN gets better results and is closest to his neighbors even if they are still better. LWLR has still the best average but WNN has a significant better standard deviation and a smallest maximum distance. To conclude, LWLR looks better in average but seems to be less good sometimes than the WNN model.

#### Time processing comparison

NN model is far away better than his neighbor in speed processing in any case : around 4 times better than LWLR model and twine better than WNN with an equal standard deviation. 

#### Forward models for Non-Stationnary environments

'NSNN' and 'NSLWLR' are modified versions of 'NN' and 'LWLR' where points are not only weighted by distance but also by the number of points that appeared after that one (gaussian with parameter sigma_t=100), to put less weight on old points and allow the learning of Non-Stationnary environments. *How to use it ?*

###Inverse models comparison


#### Calculation comparison & computation

##### Nearest Neighbor inverse model


To perform the inverse inference, the Nearest Neighbor inverse model just look in the dataset of tuples $(m, s)$, the nearest neighbor of the given $s$ motor command, and return its corresponding $m$.
It works sufficiently well in different typical robotic applications but can be very long if the dataset's size exceed $10⁵$. 

Let's see how to use it : *pas de random s?*

In [24]:
#Creation of the dataset
random_motors = environment.random_motors(n=1000)
    
sm_cls, sm_configs = sensorimotor_models['nearest_neighbor']
model = sm_cls(environment.conf, **sm_configs['default'])

for m in random_motors:
    s = environment.compute_sensori_effect(m)
    model.update(m, s)

In [25]:
#Test of the model
test = environment.random_motors(n=50)
distance = []
timer = []

for mTest in test :
    start = time.time()
    sTest_pred = model.forward_prediction(mTest)
    end = time.time()
    sTest_eff = environment.compute_sensori_effect(mTest)
    distance.append(spatial.distance.pdist([sTest_pred,sTest_eff])[0])
    timer.append(end - start)

#DRAFT bac a sable

In [ ]:
from explauto import Environment
environment = Environment.from_configuration('simple_arm', 'low_dimensional')

from explauto.sensorimotor_model import sensorimotor_models
print 'Available sensorimotor models: {}'.format(sensorimotor_models.keys())

from scipy import spatial
import numpy as np
import time

#Environment definition
from explauto.environment.environment import Environment
environment = Environment.from_configuration('simple_arm', 'mid_dimensional')

from explauto.sensorimotor_model import sensorimotor_models
from explauto import SensorimotorModel

#Creation of the dataset
random_motors = environment.random_motors(n=10000)
    
sm_cls, sm_configs = sensorimotor_models['nearest_neighbor']
model = sm_cls(environment.conf, **sm_configs['default'])

for m in random_motors:
    s = environment.compute_sensori_effect(m)
    model.update(m, s)
    
    
#Test of the model
test = environment.random_motors(n=50)
distance = []
timer = []

for mTest in test :
    start = time.time()
    sTest_pred = model.forward_prediction(mTest)
    end = time.time()
    sTest_eff = environment.compute_sensori_effect(mTest)
    distance.append(spatial.distance.pdist([sTest_pred,sTest_eff])[0])
    timer.append(end - start)
# Distance between the predictive and effectiv value
comparison = {}
comparison['NN'] = [np.mean(distance), np.std(distance) ,max(distance),
                    np.mean(timer), np.std(timer), max(timer)]
#print comparison['NN']

#Creation of the dataset
random_motors = environment.random_motors(n=10000)
    
sm_cls, sm_configs = sensorimotor_models['WNN']
model = sm_cls(environment.conf, **sm_configs['default'])

for m in random_motors:
    s = environment.compute_sensori_effect(m)
    model.update(m, s)
#Test of the model
test = environment.random_motors(n=50)
distance = []
timer = []

for mTest in test :
    start = time.time()
    sTest_pred = model.forward_prediction(mTest)
    end = time.time()
    sTest_eff = environment.compute_sensori_effect(mTest)
    distance.append(spatial.distance.pdist([sTest_pred,sTest_eff])[0])
    timer.append(end - start)

# Distance between the predictive and effectiv value
comparison['WNN'] = [np.mean(distance), np.std(distance) ,max(distance),
                    np.mean(timer), np.std(timer), max(timer)]
#print comparison['NN']
#Creation of the dataset
random_motors = environment.random_motors(n=10000)
    
sm_cls, sm_configs = sensorimotor_models['LWLR-BFGS']
model = sm_cls(environment.conf, **sm_configs['default'])

for m in random_motors:
    s = environment.compute_sensori_effect(m)
    model.update(m, s)
#Test of the model
test = environment.random_motors(n=50)
distance = []
timer = []

for mTest in test :
    start = time.time()
    sTest_pred = model.forward_prediction(mTest)
    end = time.time()
    sTest_eff = environment.compute_sensori_effect(mTest)
    distance.append(spatial.distance.pdist([sTest_pred,sTest_eff])[0])
    timer.append(end - start)
    
# Distance between the predictive and effectiv value
comparison['LWLR'] = [np.mean(distance), np.std(distance) ,max(distance),
                    np.mean(timer), np.std(timer), max(timer)]
#print comparison['NN']

for i in range(0, 6) :
    for c in comparison.keys():
        print c, i, comparison[c][i],
    print '\n'
        
#print comparison['NN']
#print comparison['WNN']
#print comparison['LWLR']